# This project consit in segmenting and clustering neigbourhood in Toronto

In [2]:

! pip install  bs4

from bs4 import BeautifulSoup


In [3]:
! pip install requests

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt


In [5]:
import requests

## Let scrap the wikipedia data using the ling provided

In [75]:
url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(url,'lxml')

In [76]:
My_table=soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [8]:
table_data = My_table.tbody.find_all("tr") 

## In the following  I am going to process data Scrapped from wikipedia and transform to a useable data format
**Here we process only data with  Borough assigned and set Neighbourhood to the corresponding Borough when not assigned**

In [83]:
# Data processing
data = {}
data['Postcode']=[]
data['Borough']=[]
data['Neighbourhood']=[]

for i in range(1,len(table_data)):
    Post=table_data[i].find_all("td")[0].text.strip()
    if table_data[i].find_all("a"):
        Bor=table_data[i].find_all("a")[0].get('title')
        if len(table_data[i].find_all("a"))>1:
            Nei=table_data[i].find_all("a")[1].get('title')
        else:
             Nei=Bor
        data['Postcode'].append(Post)
        data['Borough'].append(Bor)
        data['Neighbourhood'].append(Nei)
DataDataFrame =pd.DataFrame.from_dict(data)
DataDataFrame.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park (Toronto)
6,M9A,Queen's Park (Toronto),Queen's Park (Toronto)
7,M1B,"Scarborough, Toronto","Rouge, Toronto"
8,M1B,"Scarborough, Toronto","Malvern, Toronto"
9,M3B,North York,North York


## Let count how many time does each Postcode appear in the data set

In [84]:
counteur=DataDataFrame.Postcode.value_counts()
counteur

M9V    8
M8Y    8
M5V    7
M4V    5
M8Z    5
M9B    5
M6M    4
M1V    4
M9C    4
M9R    4
M1T    3
M2J    3
M5T    3
M3H    3
M1E    3
M1L    3
M8X    3
M8V    3
M1C    3
M6K    3
M1M    3
M6L    3
M1K    3
M5H    3
M5J    3
M1P    3
M5R    3
M2M    2
M1B    2
M6P    2
      ..
M4H    1
M4C    1
M9A    1
M2K    1
M5G    1
M3M    1
M6G    1
M9L    1
M5N    1
M7A    1
M4M    1
M4N    1
M1S    1
M3L    1
M4J    1
M1X    1
M4Y    1
M9N    1
M2H    1
M6C    1
M2N    1
M2R    1
M5C    1
M5E    1
M3A    1
M4E    1
M1W    1
M4R    1
M4A    1
M9P    1
Name: Postcode, Length: 101, dtype: int64

## As they are dupicated zip code we will combine the Neighbourhood corresponding to each PostCode as following

In [85]:
FinalEntryData=DataDataFrame.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
FinalEntryData.head()

,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough, Toronto","Rouge, Toronto,Malvern, Toronto"
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto),Rouge Hill,Port Union..."
2,M1E,"Scarborough, Toronto","Scarborough, Toronto,Morningside, Toronto,West..."
3,M1G,"Scarborough, Toronto","Woburn, Toronto"
4,M1H,"Scarborough, Toronto","Scarborough, Toronto"


In [86]:
FinalEntryData.shape

(101, 3)

# As data has been processed and cleaned,let add latitude and longitude colums


In [91]:

! pip install geocoder

# Being difficult to acces to coordinates unsing geocoder ,we will use the Geospacial_data and use pandas to load


In [92]:
!wget -q -O 'Geospatial_data.csv'  http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [93]:
Geospatial_df = pd.read_csv('Geospatial_data.csv')
Geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [94]:

# define the dataframe columns
column_names = ['Postcode','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

import geocoder # import geocoder

for index, row in FinalEntryData.iterrows():
    latitude = Geospatial_df[Geospatial_df["Postal Code"]==row['Postcode']]['Latitude'].values[0]
    longitude = Geospatial_df[Geospatial_df["Postal Code"]==row['Postcode']]['Longitude'].values[0]
    neighborhoods = neighborhoods.append({'Postcode':row['Postcode'],'Borough': row['Borough'],
                                          'Neighborhood': row['Neighbourhood'],
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)
neighborhoods.head(10)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Rouge, Toronto,Malvern, Toronto",43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497
2,M1E,"Scarborough, Toronto","Scarborough, Toronto,Morningside, Toronto,West...",43.763573,-79.188711
3,M1G,"Scarborough, Toronto","Woburn, Toronto",43.770992,-79.216917
4,M1H,"Scarborough, Toronto","Scarborough, Toronto",43.773136,-79.239476
5,M1J,"Scarborough, Toronto",Scarborough Village,43.744734,-79.239476
6,M1K,"Scarborough, Toronto","Scarborough, Toronto,Ionview,Kennedy Park, Tor...",43.727929,-79.262029
7,M1L,"Scarborough, Toronto","Clairlea,Golden Mile, Toronto,Oakridge, Toronto",43.711112,-79.284577
8,M1M,"Scarborough, Toronto","Cliffcrest,Cliffside, Toronto,Scarborough, Tor...",43.716316,-79.239476
9,M1N,"Scarborough, Toronto","Birch Cliff,Scarborough, Toronto",43.692657,-79.264848


# Now is the Clustering part

In [95]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Use geopy library to get the latitude and longitude values of Toronto.

In [96]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


# Create a map of Toronto with neighborhoods superimposed on top.

In [97]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
map_Toronto

# Define Foursquare Credentials and Version

In [98]:
CLIENT_ID = 'BF3KRSJVJ00WI2KN5ODMGWHPFKO5JEXD1XJV0TUEOH42TPRK' # your Foursquare ID
CLIENT_SECRET = 'QD31EF0JNJHJCX0FH4UOMEJDSWQ4AK5IDFP1PATDKGJGSK4W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BF3KRSJVJ00WI2KN5ODMGWHPFKO5JEXD1XJV0TUEOH42TPRK
CLIENT_SECRET:QD31EF0JNJHJCX0FH4UOMEJDSWQ4AK5IDFP1PATDKGJGSK4W


#  Let's explore the first neighborhoods in our dataframe

In [119]:
neighborhoods.loc[0, 'Neighborhood']

'Rouge, Toronto,Malvern, Toronto'

In [120]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude =neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name =neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Toronto,Malvern, Toronto are 43.806686299999996, -79.19435340000001.


# Now, let's get the top 100 venues that are in Rouge, Toronto,Malvern, Toronto within a radius of 500 meters.

In [121]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=BF3KRSJVJ00WI2KN5ODMGWHPFKO5JEXD1XJV0TUEOH42TPRK&client_secret=QD31EF0JNJHJCX0FH4UOMEJDSWQ4AK5IDFP1PATDKGJGSK4W&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [122]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e385b83216785001bcb6cf6'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [123]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

**Now we are ready to clean the json and structure it into a pandas dataframe**

In [124]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [125]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


# Let explore the Neighborhoods in Toronto

**Let's create a function to repeat the same process to all the neighborhoods in Toronto**

In [126]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [127]:
# type your answer here
Toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

Rouge, Toronto,Malvern, Toronto
Highland Creek (Toronto),Rouge Hill,Port Union, Toronto
Scarborough, Toronto,Morningside, Toronto,West Hill, Toronto
Woburn, Toronto
Scarborough, Toronto
Scarborough Village
Scarborough, Toronto,Ionview,Kennedy Park, Toronto
Clairlea,Golden Mile, Toronto,Oakridge, Toronto
Cliffcrest,Cliffside, Toronto,Scarborough, Toronto
Birch Cliff,Scarborough, Toronto
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale, Toronto,Wexford, Toronto
Agincourt, Toronto
Scarborough, Toronto,Scarborough, Toronto,Tam O'Shanter – Sullivan
Agincourt North,Scarborough, Toronto,Milliken, Ontario,Scarborough, Toronto
Scarborough, Toronto
Upper Rouge
Hillcrest Village
North York,Henry Farm,North York
Bayview Village
North York,York Mills
Newtonbrook,Willowdale, Toronto
North York
North York
Willowdale West
Parkwoods
North York
Flemingdon Park,North York
Bathurst Manor,North York,Wilson Heights, Toronto
Northwood Park,York University
CFB Toronto,North York
Downsview
North Yo

# Let's check the size of the resulting dataframe

In [128]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2198, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Toronto,Malvern, Toronto",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Scarborough, Toronto,Morningside, Toronto,West...",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Scarborough, Toronto,Morningside, Toronto,West...",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Scarborough, Toronto,Morningside, Toronto,West...",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


# Unique venues

In [129]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


#  Analyze Each Neighborhood

In [130]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot =Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [131]:
Toronto_onehot.shape

(2198, 270)

In [132]:
Toronto_grouped =Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Agincourt North,Scarborough, Toronto,Milliken,...",0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.0

In [133]:
Toronto_grouped.shape

(84, 270)

# Let's print each neighborhood along with the top 5 most common venues

In [134]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North,Scarborough, Toronto,Milliken, Ontario,Scarborough, Toronto----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2                Yoga Studio   0.0
3  Middle Eastern Restaurant   0.0
4        Monument / Landmark   0.0


----Agincourt, Toronto----
                       venue  freq
0                     Lounge  0.25
1               Skating Rink  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4  Middle Eastern Restaurant  0.00


----Alderwood, Toronto,Long Branch, Toronto----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2     Coffee Shop   0.1
3  Sandwich Place   0.1
4             Pub   0.1


----Bathurst Manor,North York,Wilson Heights, Toronto----
         venue  freq
0  Coffee Shop  0.11
1     Pharmacy  0.05
2    Gift Shop  0.05
3  Supermarket  0.05
4         Bank  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  

# Let's put that into a pandas dataframe

*First, let's write a function to sort the venues in descending order.*

In [135]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

*Now let's create the new dataframe and display the top 10 venues for each neighborhood.*

In [151]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt North,Scarborough, Toronto,Milliken,...",Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,"Agincourt, Toronto",Skating Rink,Latin American Restaurant,Lounge,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Alderwood, Toronto,Long Branch, Toronto",Pizza Place,Skating Rink,Sandwich Place,Pharmacy,Pool,Pub,Dance Studio,Coffee Shop,Gym,Gay Bar
3,"Bathurst Manor,North York,Wilson Heights, Toronto",Coffee Shop,Fried Chicken Joint,Pharmacy,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Restaurant,Diner,Sandwich Place,Shopping Mall
4,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


# Cluster Neighborhoods

*Run k-means to cluster the neighborhood into 5 clusters.*

In [152]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

*Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.*

In [153]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Scarborough, Toronto","Rouge, Toronto,Malvern, Toronto",43.806686,-79.194353,2.0,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497,1.0,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,M1E,"Scarborough, Toronto","Scarborough, Toronto,Morningside, Toronto,West...",43.763573,-79.188711,1.0,Electronics Store,Pizza Place,Intersection,Mexican Restaurant,Medical Center,Rental Car Location,Breakfast Spot,Drugstore,Donut Shop,Department Store
3,M1G,"Scarborough, Toronto","Woburn, Toronto",43.770992,-79.216917,1.0,Coffee Shop,Mexican Restaurant,Korean Restaurant,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
4,M1H,"Scarborough, Toronto","Scarborough, Toronto",43.773136,-79.239476,1.0,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Thai Restaurant,Sandwich Place,Indian Restaurant,Bank,Bakery,Caribbean Restaurant,Fried Chicken Joint


# Let visualize the final cluster

In [172]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Toronto_mergedNew=Toronto_merged.dropna(subset=['Cluster Labels'])
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_mergedNew['Latitude'],Toronto_mergedNew['Longitude'],Toronto_mergedNew['Neighborhood'],Toronto_mergedNew['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Let examine the cluster


*Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to y*

**Cluster 1**

In [173]:
Toronto_mergedNew.loc[Toronto_mergedNew['Cluster Labels'] == 0,Toronto_mergedNew.columns[[1] + list(range(5,Toronto_mergedNew.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Scarborough, Toronto",0.0,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
14,"Scarborough, Toronto",0.0,Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
25,North York,0.0,Park,Food & Drink Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Falafel Restaurant
50,Downtown Toronto,0.0,Park,Playground,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Empanada Restaurant,Donut Shop,Dance Studio
96,"York, Toronto",0.0,Park,Convenience Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Deli / Bodega


**Cluster 2**

In [174]:
Toronto_mergedNew.loc[Toronto_mergedNew['Cluster Labels'] == 1,Toronto_mergedNew.columns[[1] + list(range(5,Toronto_mergedNew.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Scarborough, Toronto",1.0,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,"Scarborough, Toronto",1.0,Electronics Store,Pizza Place,Intersection,Mexican Restaurant,Medical Center,Rental Car Location,Breakfast Spot,Drugstore,Donut Shop,Department Store
3,"Scarborough, Toronto",1.0,Coffee Shop,Mexican Restaurant,Korean Restaurant,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
4,"Scarborough, Toronto",1.0,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Thai Restaurant,Sandwich Place,Indian Restaurant,Bank,Bakery,Caribbean Restaurant,Fried Chicken Joint
6,"Scarborough, Toronto",1.0,Department Store,Coffee Shop,Bus Station,Discount Store,Convenience Store,Hobby Shop,Chinese Restaurant,Donut Shop,Dim Sum Restaurant,Diner
7,"Scarborough, Toronto",1.0,Bakery,Bus Line,Soccer Field,Intersection,Bus Station,Ice Cream Shop,Women's Store,Doner Restaurant,Discount Store,Dog Run
8,"Scarborough, Toronto",1.0,American Restaurant,Motel,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
9,"Scarborough, Toronto",1.0,Skating Rink,Café,General Entertainment,College Stadium,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
10,"Scarborough, Toronto",1.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
11,"Scarborough, Toronto",1.0,Auto Garage,Middle Eastern Restaurant,Breakfast Spot,Sandwich Place,Bakery,Shopping Mall,Accessories Store,Vietnamese Restaurant,Comic Shop,Comfort Food Restaurant


**Cluster 3**

In [175]:
Toronto_mergedNew.loc[Toronto_mergedNew['Cluster Labels'] == 2,Toronto_mergedNew.columns[[1] + list(range(5,Toronto_mergedNew.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Scarborough, Toronto",2.0,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
79,"York, Toronto",2.0,Fast Food Restaurant,Restaurant,Sandwich Place,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store


**Cluster 4**

In [176]:
Toronto_mergedNew.loc[Toronto_mergedNew['Cluster Labels'] == 3,Toronto_mergedNew.columns[[1] + list(range(5,Toronto_mergedNew.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,3.0,Cafeteria,Deli / Bodega,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


**Cluster 5**

In [177]:
Toronto_mergedNew.loc[Toronto_mergedNew['Cluster Labels'] == 4,Toronto_mergedNew.columns[[1] + list(range(5,Toronto_mergedNew.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,4.0,Tennis Court,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Deli / Bodega
